# Deepfake Detection using Deep Feature Stacking and Meta-Learning

## Implementation of the research paper:
- Paper: **Deepfake Detection using Deep Feature Stacking and Meta-Learning**
- Authors: Gourab Naskar, Sk Mohiuddin, Samir Malakar, Erik Cuevas, Ram Sarkar

This notebook implements a deepfake detection method using:
- Xception - Extracts deep features from images
- EfficientNet-B7 - Extracts additional feature representations
- Feature Selection - Ranking-based selection to reduce redundant features
- Meta-Learner (MLP) - Uses selected features to classify Real vs Fake videos

In [ ]:
# Install necessary libraries (use -q flag to suppress output)
!pip install -q tensorflow keras scikit-learn tqdm matplotlib seaborn pandas numpy
!pip install -q opencv-python-headless
!pip install -q efficientnet

# Check for GPU availability
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU'))}")

# Enable memory growth for GPU to avoid OOM errors
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print(f"Memory growth enabled on {len(physical_devices)} GPU(s)")

# Import Required Libraries
Import all the libraries needed for the project, including TensorFlow, NumPy, Pandas, and Matplotlib.

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import gc
from tqdm.notebook import tqdm

# TensorFlow and Keras imports
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.applications.xception import Xception, preprocess_input as xception_preprocess
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# For feature selection and meta-learner
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load Dataset
Load the dataset from a Google Drive link or a public URL, ensuring compatibility with Colab's file system.

In [ ]:
# Download and extract dataset
!wget https://deep-fake-dataset.s3.eu-north-1.amazonaws.com/PreprocessedDatasetV1.zip -O dataset.zip
!unzip -q dataset.zip -d /content

# Define dataset paths
train_dir = '/content/PreprocessedDatasetV1/train'
test_dir = '/content/PreprocessedDatasetV1/test'
val_dir = '/content/PreprocessedDatasetV1/validation'

# Check if directories exist
!ls -la /content/PreprocessedDatasetV1
print(f"Train directory exists: {os.path.exists(train_dir)}")
print(f"Test directory exists: {os.path.exists(test_dir)}")
print(f"Validation directory exists: {os.path.exists(val_dir)}")

# Preprocess Data
Perform data preprocessing steps such as resizing images, normalizing pixel values, and splitting the dataset into training, validation, and test sets.

In [ ]:
# Configuration
IMAGE_SIZE = 224  # Standard size for many CNNs
BATCH_SIZE = 32   # Adjust based on your GPU memory
EPOCHS = 20       # Can be reduced with early stopping
FEATURE_PERCENTAGE = 30  # Top k% features to keep (as per paper)

# Data generators with proper preprocessing for Xception and EfficientNetB7
# Training data with augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=xception_preprocess,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation and test data only need preprocessing, not augmentation
val_datagen = ImageDataGenerator(preprocessing_function=xception_preprocess)
test_datagen = ImageDataGenerator(preprocessing_function=xception_preprocess)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Check dataset sizes
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {val_generator.samples}")
print(f"Testing samples: {test_generator.samples}")
print(f"Class indices: {train_generator.class_indices}")

# Build the Model
Define the deep learning model architecture using TensorFlow or PyTorch, ensuring modularity and clarity.

In [ ]:
def create_model(model_name, img_size):
    """Create and compile the CNN model"""
    if model_name == 'xception':
        # Create the Xception model directly with the right input shape
        base_model = Xception(
            include_top=False,
            weights='imagenet',
            input_shape=(img_size, img_size, 3)
        )
        
    elif model_name == 'efficientnet':
        # Create EfficientNetB7 model directly with the right input shape
        base_model = EfficientNetB7(
            include_top=False,
            weights='imagenet',
            input_shape=(img_size, img_size, 3)
        )
    else:
        raise ValueError(f"Model {model_name} not supported")
    
    # Add custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model, base_model

# Create Xception and EfficientNetB7 models
print("Creating Xception model...")
xception_model, xception_base = create_model('xception', IMAGE_SIZE)

print("Creating EfficientNetB7 model...")
efficientnet_model, efficientnet_base = create_model('efficientnet', IMAGE_SIZE)

# Print model summaries
print("\nXception Model Summary:")
xception_model.summary()

print("\nEfficientNetB7 Model Summary:")
efficientnet_model.summary()

# Train the Model
Train the model on the preprocessed dataset, including callbacks for early stopping and saving the best model.

In [ ]:
def train_model_with_fine_tuning(model, base_model, train_gen, valid_gen, model_name, epochs=EPOCHS):
    """Train model with two-phase fine tuning strategy"""
    # Define callbacks
    checkpoint = ModelCheckpoint(
        f'{model_name}_best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
    
    early_stop = EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        mode='max',
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_accuracy', 
        factor=0.2,
        patience=3,
        min_lr=1e-6,
        mode='max',
        verbose=1
    )
    
    callbacks = [checkpoint, early_stop, reduce_lr]
    
    # Phase 1: Initial training with frozen base model
    print(f"Training {model_name} - Phase 1: Training only top layers")
    history1 = model.fit(
        train_gen,
        validation_data=valid_gen,
        epochs=epochs // 2,
        callbacks=callbacks
    )
    
    # Phase 2: Unfreeze some layers for fine-tuning
    if model_name == 'xception':
        for layer in base_model.layers[-30:]:  # Unfreeze last 30 layers
            layer.trainable = True
    else:  # efficientnet
        for layer in base_model.layers[-50:]:  # Unfreeze last 50 layers
            layer.trainable = True
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=0.0001),  # Lower learning rate for fine-tuning
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    print(f"Training {model_name} - Phase 2: Fine-tuning")
    history2 = model.fit(
        train_gen,
        validation_data=valid_gen,
        epochs=epochs // 2,
        callbacks=callbacks
    )
    
    # Combine histories
    combined_history = {}
    for key in history1.history.keys():
        combined_history[key] = history1.history[key] + history2.history[key]
    
    return model, combined_history

# Train both models with fine-tuning
print("\n==== Training Xception Model ====")
xception_model, xception_history = train_model_with_fine_tuning(
    xception_model, xception_base, train_generator, val_generator, 'xception'
)

# Clear memory before training next model
gc.collect()
tf.keras.backend.clear_session()

print("\n==== Training EfficientNetB7 Model ====")
efficientnet_model, efficientnet_history = train_model_with_fine_tuning(
    efficientnet_model, efficientnet_base, train_generator, val_generator, 'efficientnet'
)

# Plot training history for both models
def plot_training_history(history, model_name):
    """Plot training history"""
    plt.figure(figsize=(12, 5))
    
    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot training histories
plot_training_history(xception_history, 'Xception')
plot_training_history(efficientnet_history, 'EfficientNetB7')

# Feature Extraction
Create feature extractors from the trained models and extract deep features from the images.

In [ ]:
def create_feature_extractor(model_name, img_size):
    """Create a feature extractor model"""
    if model_name == 'xception':
        base_model = Xception(
            include_top=False,
            weights='imagenet',
            input_shape=(img_size, img_size, 3),
            pooling='avg'
        )
        return base_model
        
    elif model_name == 'efficientnet':
        base_model = EfficientNetB7(
            include_top=False,
            weights='imagenet',
            input_shape=(img_size, img_size, 3),
            pooling='avg'
        )
        return base_model
    else:
        raise ValueError(f"Model {model_name} not supported")

def extract_features(feature_extractor, data_generator, name):
    """Extract features from a dataset using the given extractor"""
    print(f"Extracting features using {name}...")
    features = []
    labels = []
    batch_count = 0
    total_batches = len(data_generator)
    
    for batch_x, batch_y in tqdm(data_generator, total=total_batches):
        batch_features = feature_extractor.predict(batch_x, verbose=0)
        features.append(batch_features)
        labels.append(batch_y)
        batch_count += 1
        
        # Stop when we've gone through the entire dataset
        if batch_count >= total_batches:
            break
    
    # Concatenate all batches
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    print(f"{name} features shape: {features.shape}")
    print(f"{name} labels shape: {labels.shape}")
    
    return features, labels

# Create feature extractors
print("Creating feature extractors...")
xception_extractor = create_feature_extractor('xception', IMAGE_SIZE)
efficientnet_extractor = create_feature_extractor('efficientnet', IMAGE_SIZE)

# Reset generators for feature extraction (shuffle=False to maintain order)
train_gen_features = ImageDataGenerator(preprocessing_function=xception_preprocess).flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False  # Important: keep order of samples
)

val_gen_features = ImageDataGenerator(preprocessing_function=xception_preprocess).flow_from_directory(
    val_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_gen_features = ImageDataGenerator(preprocessing_function=xception_preprocess).flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Extract features for train set
train_xception_features, train_labels = extract_features(xception_extractor, train_gen_features, 'Train Xception')

# Reset generator for EfficientNet
train_gen_features = ImageDataGenerator(preprocessing_function=xception_preprocess).flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

train_efficientnet_features, _ = extract_features(efficientnet_extractor, train_gen_features, 'Train EfficientNet')

# Extract features for validation set
val_xception_features, val_labels = extract_features(xception_extractor, val_gen_features, 'Val Xception')

# Reset generator for EfficientNet
val_gen_features = ImageDataGenerator(preprocessing_function=xception_preprocess).flow_from_directory(
    val_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

val_efficientnet_features, _ = extract_features(efficientnet_extractor, val_gen_features, 'Val EfficientNet')

# Extract features for test set
test_xception_features, test_labels = extract_features(xception_extractor, test_gen_features, 'Test Xception')

# Reset generator for EfficientNet
test_gen_features = ImageDataGenerator(preprocessing_function=xception_preprocess).flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_efficientnet_features, _ = extract_features(efficientnet_extractor, test_gen_features, 'Test EfficientNet')

# Stack features
train_features = np.concatenate([train_xception_features, train_efficientnet_features], axis=1)
val_features = np.concatenate([val_xception_features, val_efficientnet_features], axis=1)
test_features = np.concatenate([test_xception_features, test_efficientnet_features], axis=1)

print(f"Stacked feature shapes - Train: {train_features.shape}, Val: {val_features.shape}, Test: {test_features.shape}")

# Clear memory
del xception_extractor, efficientnet_extractor
gc.collect()
tf.keras.backend.clear_session()

# Feature Selection
Use XGBoost and Random Forest to select the most important features.

In [ ]:
def select_features(train_features, train_labels, val_features, test_features, feature_percentage=FEATURE_PERCENTAGE):
    """Select top k% features using RandomForest and XGBoost feature importance"""
    print(f"Starting feature selection... Selecting top {feature_percentage}% features")
    
    # Feature selection using Random Forest
    print("Training Random Forest for feature importance...")
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(train_features, train_labels)
    rf_importances = rf.feature_importances_
    
    # Feature selection using XGBoost
    print("Training XGBoost for feature importance...")
    xgb = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, 
                       eval_metric='logloss', n_jobs=-1)
    xgb.fit(train_features, train_labels)
    xgb_importances = xgb.feature_importances_
    
    # Average feature importances and select top k%
    average_importances = (rf_importances + xgb_importances) / 2
    k = int(feature_percentage * len(average_importances) / 100)
    
    # Get indices of top k% features
    top_k_indices = np.argsort(average_importances)[-k:]
    
    print(f"Selected {k} features out of {len(average_importances)} ({feature_percentage}%)")
    
    # Create a feature mask for visualization
    feature_mask = np.zeros(len(average_importances), dtype=bool)
    feature_mask[top_k_indices] = True
    
    # Visualize top feature importance
    plt.figure(figsize=(12, 6))
    plt.bar(range(k), average_importances[top_k_indices], color='skyblue')
    plt.title(f'Top {k} Feature Importance')
    plt.xlabel('Feature Index (sorted by importance)')
    plt.ylabel('Average Importance Score')
    plt.tight_layout()
    plt.show()
    
    # Apply feature selection to datasets
    train_selected = train_features[:, top_k_indices]
    val_selected = val_features[:, top_k_indices]
    test_selected = test_features[:, top_k_indices]
    
    return train_selected, val_selected, test_selected, top_k_indices

# Apply feature selection
train_features, val_features, test_features, selected_indices = select_features(
    train_features, train_labels, val_features, test_features, FEATURE_PERCENTAGE
)

# Meta-Learner
Train a Meta-Learner (MLP) to classify Real vs Fake videos.

In [ ]:
def train_meta_learner(train_features, train_labels, val_features, val_labels):
    """Train MLP meta-learner on selected features"""
    print("Training meta-learner (MLP)...")
    
    # Create MLP classifier with optimal architecture for deepfake detection
    meta_learner = Sequential([
        Dense(256, activation='relu', input_shape=(train_features.shape[1],)),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    meta_learner.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy', 
        metrics=['accuracy']
    )
    
    # Define callbacks
    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True,
        mode='max',
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.2,
        patience=3,
        min_lr=1e-6,
        mode='max',
        verbose=1
    )
    
    # Train the meta-learner
    history = meta_learner.fit(
        train_features, train_labels,
        validation_data=(val_features, val_labels),
        epochs=30,
        batch_size=32,
        callbacks=[early_stopping, reduce_lr]
    )
    
    return meta_learner, history

# Train the meta-learner
meta_learner, meta_history = train_meta_learner(train_features, train_labels, val_features, val_labels)

# Plot meta-learner training history
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(meta_history.history['accuracy'], label='Training Accuracy')
plt.plot(meta_history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Meta-Learner Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(meta_history.history['loss'], label='Training Loss')
plt.plot(meta_history.history['val_loss'], label='Validation Loss')
plt.title('Meta-Learner Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate the Model
Evaluate the model's performance on the test dataset and visualize metrics such as accuracy and loss.

In [ ]:
def evaluate_meta_learner(model, X_test, y_test):
    """Evaluate meta-learner on test set with comprehensive metrics"""
    print("Evaluating meta-learner...")
    
    # Get predictions
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    y_pred_prob = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_prob)
    
    print(f"Meta-learner Test Accuracy: {accuracy:.4f}")
    print(f"Meta-learner Test Precision: {precision:.4f}")
    print(f"Meta-learner Test Recall: {recall:.4f}")
    print(f"Meta-learner Test F1 Score: {f1:.4f}")
    print(f"Meta-learner Test ROC AUC: {auc:.4f}")
    
    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Meta-learner Confusion Matrix')
    plt.show()
    
    # Classification Report
    print("Meta-learner Classification Report:")
    print(classification_report(y_test, y_pred, target_names=['Real', 'Fake']))
    
    return accuracy, precision, recall, f1, auc

# Evaluate the model with comprehensive metrics
metrics = evaluate_meta_learner(meta_learner, test_features, test_labels)

# Compare with baseline models if available (optional)
if 'xception_model' in globals() and 'efficientnet_model' in globals():
    print("\nComparing with base models performance:")
    models = ['Meta-Learner']
    metrics_df = pd.DataFrame({
        'Model': models,
        'Accuracy': [metrics[0]],
        'Precision': [metrics[1]],
        'Recall': [metrics[2]],
        'F1 Score': [metrics[3]],
        'AUC': [metrics[4]]
    })
    
    print(metrics_df)

# Visualize Model Predictions on Validation Set
Let's visualize how our model performs on the validation set to get a better understanding of its strengths and weaknesses.

In [ ]:
# Getting validation file paths
val_file_paths = val_generator.filepaths

print("Visualizing sample predictions on validation set...")
visualize_sample_predictions(meta_learner, val_features, val_labels, val_file_paths, num_samples=5)

# Optional: You can also see more examples by increasing the number of samples
# Uncomment the line below to see 10 examples instead
# visualize_sample_predictions(meta_learner, val_features, val_labels, val_file_paths, num_samples=10)

# Visualize Model Predictions on Test Set
Let's also visualize how our model performs on the test set to assess its generalization capabilities.

In [ ]:
# Getting test file paths
test_file_paths = test_generator.filepaths

print("Visualizing sample predictions on test set...")
visualize_sample_predictions(meta_learner, test_features, test_labels, test_file_paths, num_samples=5)

# Sample Model Predictions
Visualize a few sample predictions from our model to see how well it's performing on specific examples.

In [ ]:
def visualize_sample_predictions(model, features, labels, true_file_paths, num_samples=5):
    """Visualize sample predictions from the model with original images"""
    import random
    import cv2
    from tensorflow.keras.preprocessing.image import load_img
    
    # Generate predictions for all samples
    pred_probs = model.predict(features)
    predictions = (pred_probs > 0.5).astype(int).flatten()
    
    # Find correct and incorrect predictions
    correct_indices = np.where(predictions == labels)[0]
    incorrect_indices = np.where(predictions != labels)[0]
    
    # Try to get an even mix of correct and incorrect predictions if possible
    num_correct = min(num_samples // 2 + num_samples % 2, len(correct_indices))
    num_incorrect = min(num_samples // 2, len(incorrect_indices))
    
    # If we don't have enough incorrect predictions, use more correct ones
    if num_incorrect < num_samples // 2:
        num_correct = min(num_samples - num_incorrect, len(correct_indices))
    
    # If we don't have enough correct predictions, use more incorrect ones
    if num_correct < num_samples // 2 + num_samples % 2:
        num_incorrect = min(num_samples - num_correct, len(incorrect_indices))
    
    # Randomly sample from correct and incorrect predictions
    sampled_correct = random.sample(list(correct_indices), num_correct) if num_correct > 0 else []
    sampled_incorrect = random.sample(list(incorrect_indices), num_incorrect) if num_incorrect > 0 else []
    
    # Combine the samples
    sampled_indices = sampled_correct + sampled_incorrect
    random.shuffle(sampled_indices)  # Shuffle to mix correct and incorrect predictions
    
    # Limit to the requested number of samples
    sampled_indices = sampled_indices[:num_samples]
    
    # Create a figure to display the results
    plt.figure(figsize=(12, 4 * num_samples))
    
    for i, idx in enumerate(sampled_indices):
        # Get the prediction details
        true_label = labels[idx]
        pred_label = predictions[idx]
        prob = pred_probs[idx][0]
        
        # Get the corresponding file path
        file_path = true_file_paths[idx]
        
        # Load the image
        try:
            img = load_img(file_path)
            plt.subplot(num_samples, 1, i+1)
            plt.imshow(img)
            plt.axis('off')
            
            # Set title based on prediction correctness
            if true_label == pred_label:
                result = "CORRECT"
                color = 'green'
            else:
                result = "INCORRECT"
                color = 'red'
                
            true_text = "Real" if true_label == 0 else "Fake"
            pred_text = "Real" if pred_label == 0 else "Fake"
            
            title = f"{result}: {file_path.split('/')[-2]}\nTrue: {true_text}, Predicted: {pred_text} (Confidence: {prob:.4f})"
            plt.title(title, color=color, fontsize=12)
            
        except Exception as e:
            plt.subplot(num_samples, 1, i+1)
            plt.text(0.5, 0.5, f"Error loading image: {e}", ha='center', va='center')
            plt.axis('off')
    
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.5)
    plt.show()

# Robustness Testing
Test the model's robustness to variations in brightness as mentioned in the paper.

In [ ]:
def test_model_robustness(meta_learner, selected_indices, test_dir):
    """Test model robustness with brightness variations as mentioned in the paper"""
    print("Testing model robustness to brightness variations...")
    
    # Create feature extractors
    xception_extractor = create_feature_extractor('xception', IMAGE_SIZE)
    efficientnet_extractor = create_feature_extractor('efficientnet', IMAGE_SIZE)
    
    # Create data generators with brightness variations
    brightness_variations = [0.5, 0.75, 1.0, 1.25, 1.5]  # 50%, 75%, 100%, 125%, 150%
    results = []
    
    for brightness in brightness_variations:
        print(f"Testing with brightness factor: {brightness}")
        
        # Create a test generator with brightness adjustment
        test_brightness_datagen = ImageDataGenerator(
            preprocessing_function=xception_preprocess,
            brightness_range=[brightness, brightness]
        )
        
        test_brightness_gen = test_brightness_datagen.flow_from_directory(
            test_dir,
            target_size=(IMAGE_SIZE, IMAGE_SIZE),
            batch_size=BATCH_SIZE,
            class_mode='binary',
            shuffle=False
        )
        
        # Extract features for Xception
        test_x_features, test_labels = extract_features(xception_extractor, test_brightness_gen, f'Xception Brightness-{brightness}')
        
        # Reset generator for EfficientNet
        test_brightness_gen = test_brightness_datagen.flow_from_directory(
            test_dir,
            target_size=(IMAGE_SIZE, IMAGE_SIZE),
            batch_size=BATCH_SIZE,
            class_mode='binary',
            shuffle=False
        )
        
        # Extract features for EfficientNet
        test_e_features, _ = extract_features(efficientnet_extractor, test_brightness_gen, f'EfficientNet Brightness-{brightness}')
        
        # Stack features
        test_stacked = np.concatenate([test_x_features, test_e_features], axis=1)
        
        # Apply feature selection
        # Create feature mask
        feature_mask = np.zeros(test_stacked.shape[1], dtype=bool)
        feature_mask[selected_indices] = True
        
        # Select features
        test_selected = test_stacked[:, feature_mask]
        
        # Evaluate
        y_pred = (meta_learner.predict(test_selected) > 0.5).astype("int32")
        
        # Calculate metrics
        accuracy = accuracy_score(test_labels, y_pred)
        precision = precision_score(test_labels, y_pred)
        recall = recall_score(test_labels, y_pred)
        f1 = f1_score(test_labels, y_pred)
        
        results.append({
            'brightness': brightness,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1
        })
        
        print(f"Brightness {brightness} - Accuracy: {accuracy:.4f}, F1: {f1:.4f}")
    
    # Convert results to DataFrame for easier analysis
    df_results = pd.DataFrame(results)
    
    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(df_results['brightness'], df_results['accuracy'], 'o-', label='Accuracy')
    plt.plot(df_results['brightness'], df_results['f1_score'], 's-', label='F1 Score')
    plt.plot(df_results['brightness'], df_results['precision'], '^-', label='Precision')
    plt.plot(df_results['brightness'], df_results['recall'], 'd-', label='Recall')
    plt.xlabel('Brightness Factor')
    plt.ylabel('Score')
    plt.title('Model Robustness to Brightness Variations')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Display results as a table
    print("\nRobustness Test Results:")
    print(df_results.round(4))
    
    return df_results

# Run robustness test
# Comment out the next line if you want to skip this test during initial development
robustness_results = test_model_robustness(meta_learner, selected_indices, test_dir)

# Save and Export Results
Save the trained model and export results such as predictions or performance metrics to Google Drive or a downloadable file.

In [ ]:
# First, evaluate the model to get metrics
print("Evaluating model on test set to get final metrics...")
# Get predictions
test_predictions = (meta_learner.predict(test_features) > 0.5).astype("int32")
test_pred_prob = meta_learner.predict(test_features)

# Calculate metrics
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)
auc = roc_auc_score(test_labels, test_pred_prob)

metrics = [accuracy, precision, recall, f1, auc]
print(f"Final metrics - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

# Save the trained model and feature selection indices
# Create a directory for saved models if it doesn't exist
os.makedirs('/content/saved_models', exist_ok=True)

# Save the meta-learner model
meta_learner.save('/content/saved_models/meta_learner_model.h5')

# Save the selected feature indices
np.save('/content/saved_models/selected_feature_indices.npy', selected_indices)

# Export predictions to CSV
np.savetxt('/content/saved_models/test_predictions.csv', test_predictions, delimiter=',', fmt='%d')

# Save performance metrics
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC'],
    'Score': metrics
})
metrics_df.to_csv('/content/saved_models/performance_metrics.csv', index=False)

print("Model, predictions, and metrics saved successfully to '/content/saved_models' directory")

# For Google Colab: Download files to local machine
try:
    from google.colab import files
    # Compress the saved_models directory
    !zip -r /content/saved_models.zip /content/saved_models
    # Download the zip file
    files.download('/content/saved_models.zip')
    print("\nDownload initiated for saved_models.zip")
except ImportError:
    print("\nNot running in Google Colab, files saved locally only")

# Inference on New Data
Run inference on new images or videos to detect deepfakes using our trained model.

In [ ]:
def inference_on_new_data(image_path=None, video_path=None, model_path='/content/saved_models/meta_learner_model.h5', indices_path='/content/saved_models/selected_feature_indices.npy'):
    """Function to run inference on new images or videos"""
    if image_path is None and video_path is None:
        print("Please provide either an image path or a video path")
        return
    
    # Load the meta-learner model
    try:
        from tensorflow.keras.models import load_model
        meta_learner = load_model(model_path)
        selected_indices = np.load(indices_path)
        print(f"Loaded model from {model_path} and selected features from {indices_path}")
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Please ensure the model and feature indices files exist")
        return
    
    # Create feature extractors
    xception_extractor = create_feature_extractor('xception', IMAGE_SIZE)
    efficientnet_extractor = create_feature_extractor('efficientnet', IMAGE_SIZE)
    
    if image_path:
        process_image(image_path, xception_extractor, efficientnet_extractor, meta_learner, selected_indices)
    elif video_path:
        process_video(video_path, xception_extractor, efficientnet_extractor, meta_learner, selected_indices)

def process_image(image_path, xception_extractor, efficientnet_extractor, meta_learner, selected_indices):
    """Process a single image for deepfake detection"""
    from tensorflow.keras.preprocessing.image import load_img, img_to_array
    import time
    
    print(f"Processing image: {image_path}")
    start_time = time.time()
    
    # Load and preprocess the image
    img = load_img(image_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = xception_preprocess(img_array)
    
    # Extract features
    xception_features = xception_extractor.predict(img_array, verbose=0)
    efficientnet_features = efficientnet_extractor.predict(img_array, verbose=0)
    
    # Stack features
    stacked_features = np.concatenate([xception_features, efficientnet_features], axis=1)
    
    # Apply feature selection
    feature_mask = np.zeros(stacked_features.shape[1], dtype=bool)
    feature_mask[selected_indices] = True
    selected_features = stacked_features[:, feature_mask]
    
    # Make prediction
    prediction_prob = meta_learner.predict(selected_features, verbose=0)[0][0]
    prediction = 1 if prediction_prob > 0.5 else 0
    
    # Calculate processing time
    process_time = time.time() - start_time
    
    # Display result
    label = "Fake" if prediction == 1 else "Real"
    confidence = prediction_prob if prediction == 1 else 1 - prediction_prob
    print(f"Prediction: {label} (confidence: {confidence:.4f})")
    print(f"Processing time: {process_time:.2f} seconds")
    
    # Display the image with prediction
    plt.figure(figsize=(8, 8))
    plt.imshow(load_img(image_path))
    plt.title(f"Prediction: {label} (confidence: {confidence:.4f})")
    plt.axis('off')
    plt.show()
    
    return label, confidence

def process_video(video_path, xception_extractor, efficientnet_extractor, meta_learner, selected_indices):
    """Process a video for deepfake detection"""
    import cv2
    from tqdm.notebook import tqdm
    import time
    
    print(f"Processing video: {video_path}")
    start_time = time.time()
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps
    
    print(f"Video info - Frames: {frame_count}, FPS: {fps:.2f}, Duration: {duration:.2f}s")
    
    predictions = []
    probabilities = []
    frames_to_show = []
    
    # Process frames at regular intervals (analyze ~20 frames throughout the video)
    interval = max(1, frame_count // 20)
    frames_to_process = range(0, frame_count, interval)
    
    for i in tqdm(frames_to_process, desc="Processing video frames"):
        # Set the position of the next frame to read
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        
        if not ret:
            print(f"Failed to read frame {i}")
            continue
        
        # Convert frame from BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Save a few frames to display later
        if len(frames_to_show) < 4 and i % (interval * 4) == 0:
            frames_to_show.append((i, frame_rgb.copy()))
        
        # Resize frame
        frame_resized = cv2.resize(frame_rgb, (IMAGE_SIZE, IMAGE_SIZE))
        
        # Preprocess frame
        frame_array = np.expand_dims(frame_resized, axis=0)
        frame_array = xception_preprocess(frame_array)
        
        # Extract features
        xception_features = xception_extractor.predict(frame_array, verbose=0)
        efficientnet_features = efficientnet_extractor.predict(frame_array, verbose=0)
        
        # Stack features
        stacked_features = np.concatenate([xception_features, efficientnet_features], axis=1)
        
        # Apply feature selection
        feature_mask = np.zeros(stacked_features.shape[1], dtype=bool)
        feature_mask[selected_indices] = True
        selected_features = stacked_features[:, feature_mask]
        
        # Make prediction
        prediction_prob = meta_learner.predict(selected_features, verbose=0)[0][0]
        prediction = 1 if prediction_prob > 0.5 else 0
        
        # Save results
        predictions.append(prediction)
        probabilities.append(prediction_prob)
    
    # Close the video file
    cap.release()
    
    # Calculate final prediction and metrics
    final_prediction = 1 if sum(predictions) / len(predictions) > 0.5 else 0
    final_probability = sum(probabilities) / len(probabilities)
    processing_time = time.time() - start_time
    
    # Display result
    label = "Fake" if final_prediction == 1 else "Real"
    confidence = final_probability if final_prediction == 1 else 1 - final_probability
    print(f"Video prediction: {label} (confidence: {confidence:.4f})")
    print(f"Processing time: {processing_time:.2f} seconds")
    
    # Display sample frames
    if frames_to_show:
        plt.figure(figsize=(16, 4 * len(frames_to_show) // 2))
        for idx, (frame_idx, frame) in enumerate(frames_to_show):
            plt.subplot(len(frames_to_show) // 2 + len(frames_to_show) % 2, 2, idx + 1)
            plt.imshow(frame)
            time_point = frame_idx / fps
            plt.title(f"Frame {frame_idx} (Time: {time_point:.2f}s)")
            plt.axis('off')
        plt.tight_layout()
        plt.show()
    
    # Plot predictions over time
    plt.figure(figsize=(12, 6))
    time_points = [i / fps for i in frames_to_process[:len(probabilities)]]
    plt.plot(time_points, probabilities, 'o-', markersize=4, label='Frame Predictions')
    plt.axhline(y=0.5, color='r', linestyle='--', label='Decision Threshold')
    plt.axhline(y=final_probability, color='g', linestyle='-', label='Average Prediction')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Probability of Fake')
    plt.title(f"Video Classification: {label} (confidence: {confidence:.4f})")
    plt.ylim(0, 1)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    return label, confidence

# Example usage (uncomment to use)
# inference_on_new_data(image_path='/content/PreprocessedDatasetV1/test/real/example.jpg')  # For single image
# inference_on_new_data(video_path='/content/example_video.mp4')  # For video